In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

open a database

In [2]:
string dbPath = @"BowShock_db";
var database = OpenOrCreateDatabase(dbPath);
database

{ Session Count = 3; Grid Count = 3; Path = BowShock_db }

run BowShock simulation and save to database

In [3]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
BoSSS.Solution.Application.InitMPI(num_threads:1);
var p = new XESFMain();
var TermNArrays = new int[][] { new int[] { 8,8,8,8,8,8 } };
double tALNR=1.001;

var C = XESFHardCodedControl.XDGBowShock_TwoLs_LSFromDB(
                dbPath:dbPath,
                agg: 0.4,
                numOfCellsX: 5, numOfCellsY: 16,
                initialValue: GetInitialValue.FromDBSinglePhase,
                shockLevelSet_SessionId: @"9c45ebf9-f3e0-4d1d-bf91-776bf46e4fc2",
                pointPath: @".\BowShockPoints.txt",
                shockLevelSet_Db: @".\bosss_db_levelSets.zip",
                interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
                bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
                PlotInterval:-1,
                dgDegreeEnd:3,
                dgDegreeStart:0,
                TermNs:new int[] { 8,8,8,8,8,8 } ,
                MinPIter: new int[] {30,30,20,10,10,10},
                MaxReInits: new int[] { 30, 20, 10, 5, 5, 5 },
                ReInitTols: new double[] { 0,-0.2,-3, -4},
                applyReInit:true,
                tALNRs:new double[] { tALNR, tALNR, tALNR, tALNR, tALNR, tALNR },
                gammaMin:1e-4
                );
p.Init(C);
p.RunSolverMode(); 
//p.InitializeEverything();

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [10]:
var database = OpenOrCreateDatabase(dbPath);
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
si

XDGBowShock_TwoLs	XDGBowShock_TwoLs_p3_x5_y16_agg0.4_PContinuation_LS2FluxGodunovInterface*	5/25/2024 4:16:10 PM	90dcdaaa...

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [11]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [12]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

In [13]:
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M988.5,57.1 L1041.9,57.1 M45.6,112.3 L51.7,113.7 L57.9,114.0 L64.0,115.0 L70.1,117.0 L76.3,117.8
 L82.4,118.8 L88.6,123.4 L94.7,127.4 L100.8,124.0 L107.0,132.8 L113.1,134.6 L119.2,135.2 L125.4,135.2
 L131.5,135.2 L137.6,135.2 L143.8,135.2 L149.9,135.2 L156.1,135.2 L162.2,135.2 L168.3,135.2 L174.5,135.3
 L180.6,136.5 L186.7,135.5 L192.9,136.4 L199.0,136.7 L205.1,136.7 L211.3,136.0 L217.4,136.6 L223.5,136.8
 L229.7,137.0 L235.8,137.1 L242.0,137.1 L248.1,136.3 L254.2,137.2 L260.4,136.2 L266.5,137.0 L272.6,136.9
 L278.8,137.2 L284.9,137.2 L291.0,137.4 L297.2,137.4 L303.3,137.7 L309.5,137.5 L315.6,137.4 L321.7,137.4
 L327.9,137.2 L334.0,136.9 L340.1,137.0 L346.3,137.1 L352.4,137.1 L358.5,113.9 L364.7,114.3 L370.8,114.7
 L377.0,114.0 L383.1,114.9 L389.2,115.8 L395.4,116.0 L401.5,116.0 L407.6,116.1 L413.8,116.1 L419.9,116.1
 L426.0,116.1 L432.2,116.1 L438.3,116.1 L444.4,116.1 L450.6,114.5 L456.7,109.2 L462.9,109.1 L469.0,109.1
 L475.1,109.1 L481.3,109.1 L487.4,109.1 L493.5,109.1 L499.7,109.1 L505.8,109.1 L511.9,109.1 L518.1,109.1
 L524.2,109.1 L530.4,107.3 L536.5,115.1 L542.6,78.3 L548.8,82.2 L554.9,82.2 L561.0,43.8 L567.2,43.6
 L573.3,46.5 L579.4,49.4 L585.6,50.1 L591.7,64.0 L597.9,77.6 L604.0,93.5 L610.1,105.8 L616.3,137.6
 L622.4,158.3 L628.5,159.8 L634.7,160.9 L640.8,161.0 L646.9,161.0 L653.1,161.0 L659.2,161.0 L665.3,161.0
 L671.5,161.0 L677.6,161.0 L683.8,161.0 L689.9,161.0 L696.0,161.0 L702.2,161.0 L708.3,161.0 L714.4,161.0
 L720.6,161.0 L726.7,161.0 L732.8,161.0 L739.0,161.0 L745.1,161.0 L751.3,161.0 L757.4,161.0 L763.5,161.0
 L769.7,161.0 L775.8,161.0 L781.9,161.0 L788.1,161.0 L794.2,161.0 L800.3,161.0 L806.5,161.0 L812.6,161.0
 L818.8,161.0 L824.9,161.0 L831.0,161.0 L837.2,161.0 L843.3,161.0 L849.4,161.0 L855.6,161.0 L861.7,161.0
 L867.8,161.0 L874.0,161.0 L880.1,161.0 L886.2,161.0 L892.4,161.0 L898.5,161.0 L904.7,161.0 L910.8,161.0
 L916.9,161.0 L923.1,161.0 L929.2,161.0 L935.3,161.0 L941.5,161.0 L947.6,161.0 L953.7,161.0 L959.9,161.0
 L966.0,161.0 L972.2,166.8 L978.3,79.8 L984.4,78.2 L990.6,82.6 L996.7,83.9 L1002.8,86.9 L1009.0,103.3
 L1015.1,118.7 L1021.2,157.9 L1027.4,169.2 L1033.5,169.4 L1039.7,170.0 L1045.8,170.5 L1051.9,172.2 L1058.1,172.2
 L1064.2,172.3 L1070.3,172.3 L1076.5,172.4 L1082.6,172.4 L1088.7,172.4 L1094.9,172.4 L1101.0,172.4 L1107.1,172.4
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M988.5,81.1 L1041.9,81.1 M45.6,132.0 L51.7,133.1 L57.9,133.5 L64.0,134.8 L70.1,137.8 L76.3,139.2
 L82.4,140.9 L88.6,146.7 L94.7,154.3 L100.8,147.4 L107.0,168.7 L113.1,174.2 L119.2,177.3 L125.4,177.3
 L131.5,177.3 L137.6,177.3 L143.8,177.3 L149.9,177.3 L156.1,177.3 L162.2,177.3 L168.3,177.3 L174.5,171.0
 L180.6,178.2 L186.7,177.1 L192.9,178.7 L199.0,185.8 L205.1,185.0 L211.3,179.7 L217.4,181.6 L223.5,188.7
 L229.7,191.8 L235.8,189.3 L242.0,

optional plot

In [8]:
//p.PlotShadowFields(si);

check if residual is below a threshhold

In [14]:
NUnit.Framework.Assert.IsTrue(p.res_l2<1)